# Importing Libraries

In [32]:
import cv2 as cv
import numpy as np
import pyfeats
from skimage.feature.texture import greycomatrix
from skimage.feature.texture import greycoprops
from skimage.measure import shannon_entropy
from sklearn.ensemble import RandomForestClassifier
from joblib import load

# PreProcessing Class

In [24]:
class PreProcessing:
    def __init__(self, image_name):
        self.image = self.load_image(image_name)

    def load_image(self, image_name):
        image = cv.imread(f'dataset/demo/{image_name}', cv.IMREAD_GRAYSCALE)
        return image

    def extract_roi(self, image, start, size=(32, 32)):
        roi = image[start[0]:start[0] + size[0], start[1]:start[1] + size[1]]
        mask = np.zeros(image.shape)
        mask[start[0]:start[0] + size[0], start[1]:start[1] + size[1]] = 1
        return roi, mask

    def feature_extraction(self, image, roi_pos=[(160, 230),(118, 224),(241, 151),(120, 420),(170, 300),(400, 200),(300, 120),(240, 240),(360, 160)]):

        roi_mask_arr = []
        for pos in roi_pos:
            roi_mask_arr.append(self.extract_roi(image, pos))

        # 0 45 90 135 degrees
        angles = [0, np.pi / 4, np.pi / 2, 3 * np.pi / 4]

        da_dict = {
            0: "d1_0",
            1: "d1_45",
            2: "d1_90",
            3: "d1_135",

            4: "d2_0",
            5: "d2_45",
            6: "d2_90",
            7: "d2_135",

            8: "d3_0",
            9: "d3_45",
            10: "d3_90",
            11: "d3_135",
        }

        feat_arr = []
        for roi, mask in roi_mask_arr:
            if 0 in roi or roi.shape != (32, 32) or 0 in image.shape:
                continue
            features = {}

            glcm_mtx = greycomatrix(roi, distances=[1, 2, 3], angles=angles, levels=256)
            con = greycoprops(glcm_mtx, 'contrast').flatten()
            hom = greycoprops(glcm_mtx, 'homogeneity').flatten()
            en = greycoprops(glcm_mtx, 'energy').flatten()
            corr = greycoprops(glcm_mtx, 'correlation').flatten()

            for j in range(len(da_dict)):
                features[f'contrast_{da_dict[j]}'] = con[j]
                features[f'homogeneity_{da_dict[j]}'] = hom[j]
                features[f'energy_{da_dict[j]}'] = en[j]
                features[f'correlation_{da_dict[j]}'] = corr[j]

            features[f'entropy'] = shannon_entropy(roi)

            feat, labels = pyfeats.glrlm_features(image, mask, 256)
            for i in range(len(labels)):
                features[labels[i]] = feat[i]
            #         glrlm = {l : f for l,f in zip(labels,feat)}
            #         features[f'longRunEmphasis'] = glrlm['GLRLM_LongRunEmphasis']
            #         features[f'runPercentage'] = glrlm['GLRLM_RunPercentage']

            feat_arr.append(features)

        return feat_arr

    def build_dataframe(self):
        # dataframe consists of features of 1 ROI per image
        return pd.DataFrame(self.feature_extraction(self.image))

# Model Class

In [1]:
class Model:
    def __init__(self):
        self.__models = self.load_models('dataset/models')
        self.__preprocessing = PreProcessing(image_name)
        self.__input = self.__preprocessing.build_dataframe()
    
    def load_models(folder):
        models = {
            "normal_fatty": None,
            "normal_cirrhosis": None,
            "fatty_cirrhosis": None
        }
        for name in models.keys():
            load(f"{folder}/{name}.joblib")
        return models
    
    def load_image(image_name):
        pass
    
    def predict(self):
        return self.__model.predict(self.__input)

    def report(self):
        prediction = self.predict()
        print(f"Classification is {prediction}")

# Load Model

In [45]:
model = Model('train.png')

# Image Prediction

In [46]:
model.predict()

array(['normal', 'normal', 'normal', 'fatty', 'normal', 'normal'],
      dtype=object)

In [ ]:
# y_pred = rf.predict(X_test)
# accuracy_score(y_test, y_pred)